In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import datetime as dt
from time import sleep
from tqdm import tqdm

In [2]:
seed = 0
rng = np.random.default_rng(seed)

In [3]:
# courtesy of https://twitter.com/johnspacemuller
# https://docs.google.com/spreadsheets/d/1E8Vi4P1kl9P23uCLpYOWWY7caaQh3OZmWVD6nbSSH7Q/edit#gid=454711711

df_transfermarkt_players = pd.read_csv('..\data\johnspacemuller\Transfermarkt Values 2010-2021 - tm_player_values_2010_to_2021.csv', index_col=0)
df_transfermarkt_players['player_id'] = df_transfermarkt_players['player_url'].apply(lambda x: int(x[x.rindex('/') + 1:]))
df_transfermarkt_players

,comp_name,region,country,season_start_year,squad,player_num,player_name,player_position,player_dob,player_age,player_nationality,current_club,player_height_mtrs,player_foot,date_joined,joined_from,contract_expiry,player_market_value_euro,player_url,player_id
1,Premier League,Europe,England,2010,Chelsea FC,1,Petr Cech,Goalkeeper,1982-05-20,28.0,Czech Republic,West Bromwich Albion,1.96,left,2004-07-01,Stade Rennais FC,NaN,26000000.0,https://www.transfermarkt.com/petr-cech/profil...,5658
2,Premier League,Europe,England,2010,Chelsea FC,22,Ross Turnbull,Goalkeeper,1985-01-04,25.0,England,Arsenal FC,1.93,right,2009-07-02,Middlesbrough FC,NaN,1500000.0,https://www.transfermarkt.com/ross-turnbull/pr...,29159
3,Premier League,Europe,England,2010,Chelsea FC,40,Henrique Hilário,Goalkeeper,1975-10-21,34.0,Portugal,Burton Albion,1.89,right,2006-07-01,CD Nacional,NaN,500000.0,https://www.transfermarkt.com/henrique-hilario...,13886
4,Premier League,Europe,England,2010,Chelsea FC,26,John Terry,Centre-Back,1980-12-07,29.0,England,VfL Wolfsburg,1.87,right,1999-07-01,Chelsea FC U18,NaN,34000000.0,https://www.transfermarkt.com/john-terry/profi...,3160
5,Premier League,Europe,England,2010,Chelsea FC,2,Branislav Ivanovic,Centre-Back,1984-02-22,26.0,Serbia,Zenit St. Petersburg,1.85,right,2008-01-15,Lokomotiv Moscow,NaN,21000000.0,https://www.transfermarkt.com/branislav-ivanov...,36827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39099,LaLiga,Europe,Spain,2020,Elche CF,8,Víctor Rodríguez,Left Winger,1989-07-23,31.0,Spain,NaN,1.67,right,2020-01-30,Seattle Sounders FC,2021-06-30,800000.0,https://www.transfermarkt.com/victor-rodriguez...,129753
39100,LaLiga,Europe,Spain,2020,Elche CF,17,Josan,Right Winger,1989-12-03,31.0,Spain,NaN,1.76,right,2018-01-01,Albacete Balompié,2022-06-30,1000000.0,https://www.transfermarkt.com/josan/profil/spi...,277533
39101,LaLiga,Europe,Spain,2020,Elche CF,9,Lucas Boyé,Centre-Forward,1996-02-28,25.0,Argentina,NaN,1.83,right,2020-09-21,Torino FC,2021-06-30,3500000.0,https://www.transfermarkt.com/lucas-boye/profi...,334222
39102,LaLiga,Europe,Spain,2020,Elche CF,21,Guido Carrillo,Centre-Forward,1991-05-25,30.0,Argentina,NaN,1.91,right,2020-10-05,Southampton FC,2022-06-30,2000000.0,https://www.transfermarkt.com/guido-carrillo/p...,184672


In [4]:
columns_to_keep = ['comp_name', 'season_start_year', 'squad',
       'player_num', 'player_name', 'player_position', 'player_dob',
       'player_nationality','player_height_mtrs', 'player_foot', 
       'contract_expiry', 'player_market_value_euro',
       'player_id']

In [5]:
df_brighton_players_2018 = df_transfermarkt_players[(df_transfermarkt_players['season_start_year']==2017) & (df_transfermarkt_players['squad']=='Brighton & Hove Albion')]
df_brighton_players_2018 = df_brighton_players_2018[columns_to_keep]
df_brighton_players_2018

,comp_name,season_start_year,squad,player_num,player_name,player_position,player_dob,player_nationality,player_height_mtrs,player_foot,contract_expiry,player_market_value_euro,player_id
5955,Premier League,2017,Brighton & Hove Albion,1,Mathew Ryan,Goalkeeper,1992-04-08,Australia,1.84,right,2022-06-30,8000000.0,128969
5956,Premier League,2017,Brighton & Hove Albion,26,Tim Krul,Goalkeeper,1988-04-03,Netherlands,1.93,left,NaN,2500000.0,33027
5957,Premier League,2017,Brighton & Hove Albion,-,Christian Walton,Goalkeeper,1995-11-09,England,1.95,left,2021-06-30,250000.0,208379
5958,Premier League,2017,Brighton & Hove Albion,12,Niki Mäenpää,Goalkeeper,1985-01-23,Finland,1.91,right,NaN,250000.0,12359
5959,Premier League,2017,Brighton & Hove Albion,22,Shane Duffy,Centre-Back,1992-01-01,Ireland,1.93,right,2023-06-30,12000000.0,119269
5960,Premier League,2017,Brighton & Hove Albion,5,Lewis Dunk,Centre-Back,1991-11-21,England,1.92,right,2025-06-30,10000000.0,148153
5961,Premier League,2017,Brighton & Hove Albion,4,Uwe Hünemeier,Centre-Back,1986-01-09,Germany,1.89,right,NaN,1000000.0,10503
5962,Premier League,2017,Brighton & Hove Albion,18,Connor Goldson,Centre-Back,1992-12-18,England,1.91,right,NaN,1000000.0,163656
5963,Premier League,2017,Brighton & Hove Albion,29,Markus Suttner,Left-Back,1987-04-16,Austria,1.79,left,NaN,2500000.0,31514
5964,Premier League,2017,Brighton & Hove Albion,3,Gaëtan Bong,Left-Back,1988-04-25,Cameroon,1.83,left,NaN,1500000.0,36291


In [6]:
SUMMER_2018_DATE = dt.date(2018, 6, 1)

In [7]:
base_url = 'https://www.transfermarkt.com/a/marktwertverlauf/spieler/' # "a" is a placeholder, can be anything, technically is player name but doesn't matter

In [8]:
headers = {'User-Agent': 
       'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [9]:
transfermarkt_ids = df_brighton_players_2018['player_id']
#[148153, 82873, 209212] # sample player IDs for Gross, March and Dunk

In [10]:
columns = ['player_name', 'transfermarkt_player_id', 'date', 'market_value_in_gbp']

In [11]:
def get_market_value_history_of_player(transfermarkt_player_id, maximum_date=dt.date.today(), as_pandas=False):
    tree = requests.get(base_url + str(transfermarkt_player_id), headers=headers, timeout=300)
    soup = BeautifulSoup(tree.content, 'html.parser')
    
    try:
        script = soup.find('script', text=re.compile('Highcharts.Chart')).text
        player_name = soup.find('title').text.partition(' - ')[0]
        
        chart_data = script.split("'data':[")[1]
        ending_string = '}}]'
        chart_data = chart_data[:chart_data.find(ending_string) + 1]

        delim = '},'
        market_values = [eval(e + delim[0]) for e in chart_data.split(delim) if e[:5] == "{'y':"]

        data_points_earlier_than_max_date = [data_point for data_point in market_values if dt.date.fromtimestamp(int(str(data_point['x'])[:-3])) < maximum_date]

        if data_points_earlier_than_max_date:
            market_vals_clean = [(player_name, transfermarkt_player_id, dt.date.fromtimestamp(int(str(data_point['x'])[:-3])), data_point['y']) for data_point in market_values]

            if maximum_date:
                market_vals_clean = [[record for record in market_vals_clean if record[2] < maximum_date][-1]]

            if as_pandas:
                market_vals_clean = pd.DataFrame(market_vals_clean, columns=columns)

        else:
            market_vals_clean = None

    except AttributeError:
        market_vals_clean = None

    return market_vals_clean

In [12]:
all_player_market_values = []

for id in tqdm(transfermarkt_ids):
    sleep_time = rng.uniform(1, 3)
    sleep(sleep_time)
    this_player_transfer_history = get_market_value_history_of_player(transfermarkt_player_id=id, maximum_date=SUMMER_2018_DATE, as_pandas=False)
    if this_player_transfer_history:
        all_player_market_values.extend(this_player_transfer_history)

100%|██████████| 39/39 [02:56<00:00,  4.52s/it]


In [13]:
df_all_player_market_values = pd.DataFrame(all_player_market_values, columns=columns)
df_all_player_market_values

,player_name,transfermarkt_player_id,date,market_value_in_gbp
0,Mathew Ryan,128969,2018-05-28,8000000
1,Tim Krul,33027,2018-05-28,2500000
2,Christian Walton,208379,2017-06-28,250000
3,Niki Mäenpää,12359,2018-05-28,250000
4,Shane Duffy,119269,2018-05-28,12000000
5,Lewis Dunk,148153,2018-05-28,10000000
6,Uwe Hünemeier,10503,2018-05-28,1000000
7,Connor Goldson,163656,2018-05-28,1000000
8,Markus Suttner,31514,2018-05-28,2500000
9,Gaëtan Bong,36291,2018-05-28,1500000
